In [ ]:
import os
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

In [3]:
# Fill in the following values
PROVIDER = "github" # GitHub provider
MODEL = "vertex_ai/gemini-1.5-flash"

# Access the environment variables from the .env file
USER_TOKEN = os.environ.get('USER_TOKEN')  # GitHub user token - based on a each user account
PROJECT_ID = os.environ.get('PROJECT_ID')  # this is tied to your personal gcp's application_default_credentials.json

In [7]:
from pr_agent import cli
from pr_agent.config_loader import get_settings


def main(url: str, command: str = "review"):
    """
    Args:
        PR_URL (str): PR URL, for example 'https://github.com/Codium-ai/pr-agent/pull/809'
        COMMAND (str): Command to run (e.g. '/review', '/describe', '/ask="What is the purpose of this PR?"', ...). Defaults to "/review".
    """
    # Setting the configurations
    get_settings().set("CONFIG.git_provider", PROVIDER)
    get_settings().set("github.user_token", USER_TOKEN)

    # Set authentication    
    get_settings().set("vertexai.vertex_project", PROJECT_ID)
    get_settings().set("vertexai.vertex_location", None)
    
    # Set models
    get_settings().set("config.model", MODEL)
    get_settings().set("config.model_turbo", MODEL)
    get_settings().set("config.fallback_models",MODEL)
    
    # Additional instructions
    instructions = """\
        (1) Review the comments and only provide suggestions to the comments and do not remove any
        (2) Recommend code syntax with the best practices or suggest more efficient code practices based on the programming language used in the PR
    """
    get_settings().set("pr_code_suggestions.extra_instructions", instructions)
    get_settings().set("pr_reviewer.extra_instructions", instructions)
    
    # Run the command. Feedback will appear in GitHub PR comments
    cli.run_command(url, f"/{command}")

In [ ]:
# PR URL, for example 'https://github.com/Codium-ai/pr-agent/pull/809'
PR_URL = ""

# Run pr-agent
# command = ['review', 'improve', 'describe', 'ask ...']
main(PR_URL, "review")
main(PR_URL, "improve")